In [1]:
import sys
import numpy             as     np
import pylab             as     pl
import astropy.io.fits   as     fits

from   astropy.table     import Table
from   matplotlib.pyplot import figure

In [2]:
sys.path.append('/global/homes/m/mjwilson/desi/BGS/lumfn/py/lumfn/')
sys.path.append('/global/homes/m/mjwilson/desi/BGS/lumfn/py/')

In [3]:
from   schechter         import schechter
from   SV3.define_sample import define_sample
from   ref_gmr           import tmr_reference_gmr
from   ajs_kcorr         import ajs_kcorr
from   mxxl_ecorr        import mxxl_ecorr

In [ ]:
version = 0.5

In [ ]:
bright_merge_obs = Table.read('/global/cscratch1/sd/mjwilson/desi/BGS/lumfn/MXXL/bright_v{:.1f}.fits'.format(version))
bright_merge_obs

In [ ]:
pl.plot(bright_merge_obs['RA'], bright_merge_obs['DEC'], marker=',', lw=0.0)

In [ ]:
dz = 0.025
_  = pl.hist(bright_merge_obs['Z'], bins=np.arange(0.0, 1.0, dz), histtype='step')

pl.xlabel('Z')
pl.ylabel('N per $\Delta Z={:0.3f}$'.format(dz))

In [ ]:
dr = 0.025
_  = pl.hist(bright_merge_obs['RMAG_DRED'], bins=np.arange(12.0, 20.0, dr), histtype='step')

pl.xlabel('r')
pl.ylabel('N per $\Delta r={:0.3f}$'.format(dr))

pl.axvline(19.5, c='k', lw=0.5)
pl.axvline(19.8, c='k', lw=0.5)

In [ ]:
pl.plot(bright_merge_obs['MRH'], bright_merge_obs['REFGMR0P1'], marker=',', lw=0.0, c='k')

Ms    = np.arange(-23., -18., 0.01)

blues = 0.620 - 0.110 * (Ms + 20.)
brms  = 0.120 + 0.020 * (Ms + 20.)

reds  = 0.932 - 0.032 * (Ms + 20.)
rrms  = 0.070 + 0.010 * (Ms + 20.)

pl.plot(Ms, blues,      c='cyan')
pl.plot(Ms, blues+brms, c='cyan', alpha=0.45)
pl.plot(Ms, blues-brms, c='cyan', alpha=0.45)

pl.plot(Ms,  reds,      c='red')
pl.plot(Ms,  reds+rrms, c='red', alpha=0.45)
pl.plot(Ms,  reds-rrms, c='red', alpha=0.45)

pl.xlim(-23., -18.)
pl.ylim(0.0, 1.2)

pl.xlabel(r'$M_r$')
pl.ylabel('Ref. $(g-r)$')

In [ ]:
derived = Table.read('/global/cscratch1/sd/mjwilson/desi/BGS/lumfn/MXXL/bright_derived_v{:.1f}.fits'.format(version))
derived

In [ ]:
# bright_merge_obs = bright_merge_obs[:len(derived)]

In [ ]:
isin = derived['INSAMPLE'].astype(bool)

derived = derived[isin]
bright_merge_obs = bright_merge_obs[isin]

In [ ]:
np.mean(isin)

In [ ]:
len(derived), len(bright_merge_obs)

In [ ]:
_ = pl.hist(derived['GMR_DRED'], histtype='step', bins=np.arange(-0.4, 1.2, 0.1), label=r'$(g-r)$')
_ = pl.hist(bright_merge_obs['REFGMR0P1'], histtype='step', bins=np.arange(-0.4, 1.2, 0.1), label='Ref. $(g-r)$')

pl.xlabel(r'$(g-r)$')
pl.legend(loc=2)

In [ ]:
pl.scatter(bright_merge_obs['MRH'], (derived['MRH'] / bright_merge_obs['MRH']) - 1., marker='.', lw=0.0, c=bright_merge_obs['Z'], s=1)

pl.xlabel('AJS MRH')
pl.ylabel('(MJW / AJS) - 1 ')

pl.xlim(-23.5, -12.)
pl.ylim(-0.025, 0.015)

pl.colorbar(label='Z')

In [ ]:
zs = np.arange(0.0, 0.6, 0.01)

pl.plot(zs, zs, c='k', lw=0.5)
pl.plot(bright_merge_obs['Z'], derived['ZMAX'], marker=',', lw=0.0)

pl.xlim(0.0, 0.6)
pl.ylim(0.0, 0.6)

pl.xlabel(r'$z$')
pl.ylabel(r'$zmax$')

pl.axvline(0.5, c='k', lw=0.5)

In [ ]:
for zcut in np.arange(0.0, 0.6, 0.1):
    in_zcut = (bright_merge_obs['Z'] > zcut) & (bright_merge_obs['Z'] < zcut + 0.1)
    
    zcut_sample = bright_merge_obs[in_zcut]
    
    pl.hist(zcut_sample['MRH'], bins=np.arange(-24.0, -16.0, 0.5), histtype='step', label='{:.1f} < z < {:.1f}'.format(zcut, zcut + 0.1))
    
pl.legend(loc=1, frameon=False)
pl.xlabel(r'$M_r - 5 \log_{10}|h|$')

In [ ]:
derived['PHIKERNEL']      = derived['IVMAX']
derived['PHIKERNEL']     /= 0.01

In [ ]:
derived['MRH'].min(), derived['MRH'].max()

In [ ]:
len(derived), len(bright_merge_obs)

In [ ]:
derived['MRH_IDX']        = np.digitize(derived['MRH'], bins=np.arange(-24.0, -10.0, 0.5))

derived_grouped           = derived.group_by(['MRH_IDX'])
derived_grouped['NGROUP'] = 1 

derived_addbinned         = derived_grouped['MRH', 'PHIKERNEL', 'NGROUP'].groups.aggregate(np.add)
derived_mnbinned          = derived_grouped['MRH', 'PHIKERNEL'].groups.aggregate(np.mean)

derived_addbinned['MRH']  = derived_mnbinned['MRH']

isin                      = derived_addbinned['NGROUP'] > 10

## 
Ms = np.arange(-24., -16., 0.01)

pl.plot(derived_mnbinned['MRH'][isin], np.log10(derived_addbinned['PHIKERNEL'][isin]), marker='.', c='k', lw=0.1)
pl.plot(derived_mnbinned['MRH'][isin] + mxxl_ecorr(0.3), np.log10(derived_addbinned['PHIKERNEL'][isin]), marker='.', c='k', lw=0.1, alpha=0.3)

pl.plot(Ms, np.log10(schechter(Ms)), c='c', lw=1., label='GAMA')
pl.plot(Ms, np.log10(schechter(Ms, named_type='Blanton')), c='magenta', lw=1., label='Blanton')

pl.xlim(-23.5, -16.0)
pl.ylim(-6.1,  -1.50)

pl.xlabel(r'$M_r - 5\log_{10}|h|$')
pl.ylabel(r'$\Phi(M_r - 5\log_{10}|h|)$')

pl.title('BGS Bright SV3 (v{:.1f})'.format(version))

pl.legend(frameon=False, loc=2)

# Done.